# Data Tools Dashboard
**Various tools for data manipulation and conversion**

Created: January 2026 | KIT

In [1]:
# Import required libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
import sys
import base64
import io
import zipfile
from datetime import datetime


def _ensure_bytes(content):
    """Normalize uploaded content to bytes for downstream processing"""
    if isinstance(content, memoryview):
        return content.tobytes()
    if isinstance(content, bytearray):
        return bytes(content)
    return content

# Add current directory to path
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Import converter modules
try:
    from iv_converter_module import process_files
    from uvvis_merger_module import process_uvvis_files
    from jv_organizer_module import process_zip_file as organize_jv_files
    from eln_renamer_module import process_files as rename_files_eln
    print("✅ All modules imported successfully!")
except ImportError as e:
    print(f"⚠️ Import warning: {e}")
    print("   This is normal if running for the first time.")

✅ All modules imported successfully!


In [2]:
# IV Converter Tool
class IVConverterTool:
    """Tool for converting IV measurement files"""
    
    def __init__(self):
        self.uploaded_files = {}
        self.file_uploader = widgets.FileUpload(
            accept='.csv,.zip',
            multiple=True,
            description='Select Files',
            layout=widgets.Layout(width='200px')
        )
        self.create_widgets()
    
    def create_widgets(self):
        """Create UI widgets"""
        # Title
        self.title = widgets.HTML(
            value="<h3>📊 IV File Converter (JV Split)</h3><p>Convert Puri JV measurement files (_ivraw.csv) to old LTI format</p>"
        )
        
        # File uploader (button only)
        self.file_uploader.observe(self._on_upload, names='value')
        
        # File list display
        self.file_list = widgets.HTML(
            value="<i>No files uploaded yet</i>",
            layout=widgets.Layout(min_height='60px', max_height='150px', overflow='auto', padding='10px')
        )
        
        # Clear button
        self.clear_button = widgets.Button(
            description='Clear Files',
            button_style='warning',
            icon='trash',
            layout=widgets.Layout(width='150px')
        )
        self.clear_button.on_click(self._on_clear)
        
        # Convert button
        self.convert_button = widgets.Button(
            description='Convert Files',
            button_style='primary',
            icon='cog',
            layout=widgets.Layout(width='150px'),
            disabled=True
        )
        self.convert_button.on_click(self._on_convert)
        
        # Status output
        self.status_output = widgets.Output(
            layout=widgets.Layout(
                border='1px solid #ddd',
                padding='10px',
                min_height='80px',
                max_height='300px',
                overflow='auto'
            )
        )
        
        # Download link placeholder
        self.download_link = widgets.HTML(value="")
    
    def _iter_uploaded_files(self, file_value):
        if not file_value:
            return []
        if isinstance(file_value, dict):
            return [
                (name, _ensure_bytes(info.get('content')))
                for name, info in file_value.items()
                if isinstance(info, dict)
            ]
        return [
            (info.get('name'), _ensure_bytes(info.get('content')))
            for info in file_value
            if isinstance(info, dict) and info.get('name')
        ]
    
    def _on_upload(self, change):
        """Handle file upload"""
        if not change['new']:
            return
        
        for filename, content in self._iter_uploaded_files(change['new']):
            if not filename:
                continue
            
            # Handle zip files
            if filename.lower().endswith('.zip'):
                try:
                    with zipfile.ZipFile(io.BytesIO(content), 'r') as zip_ref:
                        for file_info in zip_ref.filelist:
                            if file_info.filename.lower().endswith(('_ivraw.csv', '.jv.csv', '.csv')):
                                extracted_name = os.path.basename(file_info.filename)
                                self.uploaded_files[extracted_name] = zip_ref.read(file_info)
                except Exception as e:
                    with self.status_output:
                        print(f"❌ Error reading zip file {filename}: {e}")
            
            # Handle CSV files
            elif filename.lower().endswith(('_ivraw.csv', '.jv.csv', '.csv')):
                self.uploaded_files[filename] = content
        
        self._update_file_list()
        self.convert_button.disabled = len(self.uploaded_files) == 0
    
    def _update_file_list(self):
        """Update the file list display"""
        if not self.uploaded_files:
            self.file_list.value = "<i>No files uploaded yet</i>"
            return
        
        file_items = []
        for idx, filename in enumerate(self.uploaded_files.keys(), 1):
            file_items.append(f"<li>{idx}. {filename}</li>")
        
        self.file_list.value = (
            f"<b>📁 Uploaded files ({len(self.uploaded_files)}):</b>"
            f"<ul style='margin-top: 5px;'>{''.join(file_items)}</ul>"
        )
    
    def _on_clear(self, button):
        """Clear all uploaded files"""
        self.uploaded_files = {}
        if isinstance(self.file_uploader.value, dict):
            self.file_uploader.value = {}
        else:
            self.file_uploader.value = ()
        self._update_file_list()
        self.convert_button.disabled = True
        self.download_link.value = ""
        with self.status_output:
            clear_output()
    
    def _on_convert(self, button):
        """Convert uploaded files"""
        with self.status_output:
            clear_output(wait=True)
            print("🔄 Converting files...")
        
        try:
            from iv_converter_module import process_files
            
            # Process files
            zip_content, num_processed = process_files(self.uploaded_files)
            
            if num_processed == 0:
                with self.status_output:
                    clear_output(wait=True)
                    print("⚠️ No files were processed. Check file format.")
                return
            
            # Create download link
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"converted_jv_files_{timestamp}.zip"
            
            b64_data = base64.b64encode(zip_content).decode()
            download_html = f'''
            <div style="margin: 15px 0; padding: 15px; background: #e8f5e9; border-radius: 5px; border-left: 4px solid #4caf50;">
                <h4 style="margin-top: 0; color: #2e7d32;">✅ Conversion Complete!</h4>
                <p>Processed <b>{num_processed}</b> output files from <b>{len(self.uploaded_files)}</b> input files.</p>
                <a href="data:application/zip;base64,{b64_data}" download="{filename}" 
                   style="display: inline-block; padding: 10px 20px; background: #4caf50; color: white; 
                          text-decoration: none; border-radius: 4px; font-weight: bold;">
                    📥 Download {filename}
                </a>
            </div>
            '''
            
            self.download_link.value = download_html
            
            with self.status_output:
                clear_output(wait=True)
                print(f"✅ Successfully converted {len(self.uploaded_files)} input files")
                print(f"📦 Generated {num_processed} output files")
                print(f"⬇️ Click the download button above to get your results")
        
        except Exception as e:
            with self.status_output:
                clear_output(wait=True)
                print(f"❌ Error during conversion: {e}")
                import traceback
                traceback.print_exc()
    
    def get_widget(self):
        """Return the complete widget"""
        return widgets.VBox([
            self.title,
            widgets.HTML("<hr style='margin: 10px 0;'>"),
            widgets.HTML("<b>1️⃣ Upload Files</b><br><i>Select CSV files or a ZIP archive</i>"),
            self.file_uploader,
            self.file_list,
            self.clear_button,
            widgets.HTML("<b>2️⃣ Convert and Download</b>"),
            self.convert_button,
            self.download_link,
            widgets.HTML("<b>Status:</b>"),
            self.status_output
        ], layout=widgets.Layout(
            padding='20px',
            border='2px solid #1976d2',
            border_radius='8px'
        ))


print("✅ IV Converter Tool class loaded")

✅ IV Converter Tool class loaded


In [3]:
# JV File Organizer Tool
class JVOrganizerTool:
    """Tool for organizing and renaming JV measurement files"""
    
    def __init__(self):
        self.uploaded_files = {}
        self.file_uploader = widgets.FileUpload(
            accept='.csv,.zip',
            multiple=True,
            description='Select Files',
            layout=widgets.Layout(width='200px')
        )
        self.create_widgets()
    
    def create_widgets(self):
        """Create UI widgets"""
        # Title
        self.title = widgets.HTML(
            value="<h3>📋 JV File Organizer</h3><p>Organize and rename JV measurement files (remove illu prefix, apply px naming scheme)</p>"
        )
        
        # File uploader (button only)
        self.file_uploader.observe(self._on_upload, names='value')
        
        # File list display
        self.file_list = widgets.HTML(
            value="<i>No files uploaded yet</i>",
            layout=widgets.Layout(min_height='60px', max_height='150px', overflow='auto', padding='10px')
        )
        
        # Cycle selection
        self.cycle_selector = widgets.IntSlider(
            value=0,
            min=0,
            max=9,
            step=1,
            description='Cycle to Keep:',
            layout=widgets.Layout(width='400px')
        )
        
        # Preserve cycle checkbox
        self.preserve_cycle_checkbox = widgets.Checkbox(
            value=False,
            description='Preserve cycle information in filenames',
            indent=False
        )
        
        # Clear button
        self.clear_button = widgets.Button(
            description='Clear Files',
            button_style='warning',
            icon='trash',
            layout=widgets.Layout(width='150px')
        )
        self.clear_button.on_click(self._on_clear)
        
        # Organize button
        self.organize_button = widgets.Button(
            description='Organize Files',
            button_style='primary',
            icon='cog',
            layout=widgets.Layout(width='150px'),
            disabled=True
        )
        self.organize_button.on_click(self._on_organize)
        
        # Status output
        self.status_output = widgets.Output(
            layout=widgets.Layout(
                border='1px solid #ddd',
                padding='10px',
                min_height='80px',
                max_height='300px',
                overflow='auto'
            )
        )
        
        # Download link placeholder
        self.download_link = widgets.HTML(value="")
    
    def _iter_uploaded_files(self, file_value):
        if not file_value:
            return []
        if isinstance(file_value, dict):
            return [
                (name, _ensure_bytes(info.get('content')))
                for name, info in file_value.items()
                if isinstance(info, dict)
            ]
        return [
            (info.get('name'), _ensure_bytes(info.get('content')))
            for info in file_value
            if isinstance(info, dict) and info.get('name')
        ]
    
    def _on_upload(self, change):
        """Handle file upload"""
        if not change['new']:
            return
        
        for filename, content in self._iter_uploaded_files(change['new']):
            if not filename:
                continue
            
            # Handle zip files
            if filename.lower().endswith('.zip'):
                try:
                    with zipfile.ZipFile(io.BytesIO(content), 'r') as zip_ref:
                        for file_info in zip_ref.filelist:
                            if file_info.filename.lower().endswith(('.csv', '.jv')):
                                extracted_name = os.path.basename(file_info.filename)
                                self.uploaded_files[extracted_name] = zip_ref.read(file_info)
                except Exception as e:
                    with self.status_output:
                        print(f"❌ Error reading zip file {filename}: {e}")
            
            # Handle CSV files
            elif filename.lower().endswith(('.csv', '.jv')):
                self.uploaded_files[filename] = content
        
        self._update_file_list()
        self.organize_button.disabled = len(self.uploaded_files) == 0
    
    def _update_file_list(self):
        """Update the file list display"""
        if not self.uploaded_files:
            self.file_list.value = "<i>No files uploaded yet</i>"
            return
        
        file_items = []
        for idx, filename in enumerate(self.uploaded_files.keys(), 1):
            file_items.append(f"<li>{idx}. {filename}</li>")
        
        self.file_list.value = (
            f"<b>📁 Uploaded files ({len(self.uploaded_files)}):</b>"
            f"<ul style='margin-top: 5px;'>{''.join(file_items)}</ul>"
        )
    
    def _on_clear(self, button):
        """Clear all uploaded files"""
        self.uploaded_files = {}
        if isinstance(self.file_uploader.value, dict):
            self.file_uploader.value = {}
        else:
            self.file_uploader.value = ()
        self._update_file_list()
        self.organize_button.disabled = True
        self.download_link.value = ""
        with self.status_output:
            clear_output()
    
    def _on_organize(self, button):
        """Organize and rename files"""
        with self.status_output:
            clear_output(wait=True)
            print("🔄 Organizing files...")
        
        try:
            from jv_organizer_module import process_files
            
            # Get parameters
            cycle_to_keep = int(self.cycle_selector.value)
            preserve_cycle = self.preserve_cycle_checkbox.value
            
            # Process files
            zip_content, num_processed, errors = process_files(
                self.uploaded_files,
                cycle_to_keep=cycle_to_keep,
                preserve_cycle=preserve_cycle
            )
            
            if zip_content is None or num_processed == 0:
                with self.status_output:
                    clear_output(wait=True)
                    print("⚠️ No files were processed. Check file format.")
                    if errors:
                        for error in errors:
                            print(f"  Error: {error}")
                return
            
            # Create download link
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"organized_jv_files_{timestamp}.zip"
            
            b64_data = base64.b64encode(zip_content).decode()
            download_html = f'''
            <div style="margin: 15px 0; padding: 15px; background: #e8f5e9; border-radius: 5px; border-left: 4px solid #4caf50;">
                <h4 style="margin-top: 0; color: #2e7d32;">✅ Organization Complete!</h4>
                <p>Processed <b>{num_processed}</b> files with the following changes:</p>
                <ul style="margin: 10px 0; padding-left: 20px;">
                    <li>Removed "_illu" suffix from filenames</li>
                    <li>Applied px naming scheme (_01_C → .px1_C, etc.)</li>
                    <li>Renamed .csv to .jv.csv</li>
                    <li>Organized files into folders (MaxPowerPointTracking, Soak)</li>
                </ul>
                <a href="data:application/zip;base64,{b64_data}" download="{filename}" 
                   style="display: inline-block; padding: 10px 20px; background: #4caf50; color: white; 
                          text-decoration: none; border-radius: 4px; font-weight: bold;">
                    📥 Download {filename}
                </a>
            </div>
            '''
            
            self.download_link.value = download_html
            
            with self.status_output:
                clear_output(wait=True)
                print(f"✅ Successfully organized {len(self.uploaded_files)} input files")
                print(f"📦 Generated {num_processed} output files")
                print(f"⬇️ Click the download button above to get your results")
                if errors:
                    print(f"\n⚠️ {len(errors)} warning(s):")
                    for error in errors:
                        print(f"  • {error}")
        
        except Exception as e:
            with self.status_output:
                clear_output(wait=True)
                print(f"❌ Error during organization: {e}")
                import traceback
                traceback.print_exc()
    
    def get_widget(self):
        """Return the complete widget"""
        return widgets.VBox([
            self.title,
            widgets.HTML("<hr style='margin: 10px 0;'>"),
            widgets.HTML("<b>1️⃣ Upload Files</b><br><i>Select CSV files or a ZIP archive</i>"),
            self.file_uploader,
            self.file_list,
            self.clear_button,
            widgets.HTML("<b>2️⃣ Configure Settings</b>"),
            self.cycle_selector,
            self.preserve_cycle_checkbox,
            widgets.HTML("<b>3️⃣ Process Files</b>"),
            self.organize_button,
            self.download_link,
            widgets.HTML("<b>Status:</b>"),
            self.status_output
        ], layout=widgets.Layout(
            padding='20px',
            border='2px solid #ff9800',
            border_radius='8px'
        ))


print("✅ JV File Organizer Tool class loaded")

✅ JV File Organizer Tool class loaded


In [ ]:
# ELN Renamer Tool
class ELNRenamerTool:
    """Tool for renaming files according to ELN naming schema"""
    
    def __init__(self):
        self.uploaded_files = {}
        self.create_widgets()
    
    def create_widgets(self):
        """Create UI widgets"""
        # Title
        self.title = widgets.HTML(
            value="<h3>📝 ELN File Renamer</h3>"
            "<p>Rename files according to ELN schema: KIT_kuerzel_datum_name_0_i.pxCycle.prozessart.csv</p>"
        )
        
        # File uploader
        self.file_uploader = widgets.FileUpload(
            accept='.csv,.zip',
            multiple=True,
            description='Upload Files',
            layout=widgets.Layout(width='100%')
        )
        self.file_uploader.observe(self._on_upload, names='value')
        
        # File list display
        self.file_list = widgets.HTML(
            value="<i>No files uploaded yet</i>",
            layout=widgets.Layout(min_height='60px', max_height='150px', overflow='auto', padding='10px')
        )
        
        # Settings section
        self.kuerzel_input = widgets.Text(
            value='',
            placeholder='e.g., RaPe, DaBa, ThFe',
            description='Your name:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )
        
        self.date_input = widgets.Text(
            value=datetime.now().strftime("%Y%m%d"),
            placeholder='YYYYMMDD',
            description='Date:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )
        
        self.prozessart_input = widgets.Text(
            value='',
            placeholder='e.g., jv, eqe, abspl',
            description='Process type:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )
        
        self.auto_date_checkbox = widgets.Checkbox(
            value=False,
            description='Use oldest file date',
            indent=False
        )
        
        # Clear button
        self.clear_button = widgets.Button(
            description='Clear Files',
            button_style='warning',
            icon='trash',
            layout=widgets.Layout(width='150px')
        )
        self.clear_button.on_click(self._on_clear)
        
        # Rename button
        self.rename_button = widgets.Button(
            description='Rename Files',
            button_style='primary',
            icon='edit',
            layout=widgets.Layout(width='150px'),
            disabled=True
        )
        self.rename_button.on_click(self._on_rename)
        
        # Status output
        self.status_output = widgets.Output(
            layout=widgets.Layout(
                border='1px solid #ddd',
                padding='10px',
                min_height='80px',
                max_height='300px',
                overflow='auto'
            )
        )
        
        # Download link placeholder
        self.download_link = widgets.HTML(value="")
    
    def _on_upload(self, change):
        """Handle file upload"""
        if not change['new']:
            return
        
        for uploaded_file in change['new']:
            filename = uploaded_file['name']
            content = _ensure_bytes(uploaded_file['content'])
            
            # Handle zip files
            if filename.lower().endswith('.zip'):
                try:
                    with zipfile.ZipFile(io.BytesIO(content), 'r') as zip_ref:
                        for file_info in zip_ref.filelist:
                            if file_info.filename.lower().endswith(('.csv', '.dat')):
                                extracted_name = os.path.basename(file_info.filename)
                                self.uploaded_files[extracted_name] = zip_ref.read(file_info)
                except Exception as e:
                    with self.status_output:
                        print(f"❌ Error reading zip file {filename}: {e}")
            
            # Handle CSV files
            elif filename.lower().endswith(('.csv', '.dat')):
                self.uploaded_files[filename] = content
        
        self._update_file_list()
        self.rename_button.disabled = len(self.uploaded_files) == 0
    
    def _update_file_list(self):
        """Update the file list display"""
        if not self.uploaded_files:
            self.file_list.value = "<i>No files uploaded yet</i>"
            return
        
        file_items = []
        for idx, filename in enumerate(self.uploaded_files.keys(), 1):
            file_items.append(f"<li>{idx}. {filename}</li>")
        
        self.file_list.value = (
            f"<b>📁 Uploaded files ({len(self.uploaded_files)}):</b>"
            f"<ul style='margin-top: 5px;'>{''.join(file_items)}</ul>"
        )
    
    def _on_clear(self, button):
        """Clear all uploaded files"""
        self.uploaded_files = {}
        self.file_uploader.value = ()
        self._update_file_list()
        self.rename_button.disabled = True
        self.download_link.value = ""
        with self.status_output:
            clear_output()
    
    def _on_rename(self, button):
        """Rename files according to ELN schema"""
        with self.status_output:
            clear_output(wait=True)
            print("🔄 Renaming files...")
        
        try:
            from eln_renamer_module import process_files
            
            # Get parameters
            kuerzel = self.kuerzel_input.value.strip()
            datum = self.date_input.value.strip()
            prozessart = self.prozessart_input.value.strip()
            use_auto_date = self.auto_date_checkbox.value
            
            # Validate
            if not kuerzel:
                with self.status_output:
                    clear_output(wait=True)
                    print("⚠️ Please enter your name (Kürzel)")
                return
            
            if not prozessart:
                with self.status_output:
                    clear_output(wait=True)
                    print("⚠️ Please enter process type (e.g., jv, eqe)")
                return
            
            # Process files
            zip_content, num_processed, errors = process_files(
                self.uploaded_files,
                kuerzel=kuerzel,
                datum=datum,
                prozessart=prozessart,
                use_auto_date=use_auto_date
            )
            
            if zip_content is None or num_processed == 0:
                with self.status_output:
                    clear_output(wait=True)
                    print("⚠️ No files were processed.")
                    if errors:
                        for error in errors:
                            print(f"  Error: {error}")
                return
            
            # Create download link
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"renamed_files_{timestamp}.zip"
            
            b64_data = base64.b64encode(zip_content).decode()
            download_html = f'''
            <div style="margin: 15px 0; padding: 15px; background: #e8f5e9; border-radius: 5px; border-left: 4px solid #4caf50;">
                <h4 style="margin-top: 0; color: #2e7d32;">✅ Renaming Complete!</h4>
                <p>Successfully renamed <b>{num_processed}</b> files.</p>
                <p style="font-size: 12px; color: #666;">
                    <b>Schema applied:</b> KIT_{kuerzel}_{datum}_{{name}}_0_{{i}}.px{{pixel}}Cycle_{{cycle}}.{prozessart}.csv
                </p>
                <a href="data:application/zip;base64,{b64_data}" download="{filename}" 
                   style="display: inline-block; padding: 10px 20px; background: #4caf50; color: white; 
                          text-decoration: none; border-radius: 4px; font-weight: bold;">
                    📥 Download {filename}
                </a>
            </div>
            '''
            
            self.download_link.value = download_html
            
            with self.status_output:
                clear_output(wait=True)
                print(f"✅ Successfully renamed {num_processed} files")
                print(f"⬇️ Click the download button above to get your results")
                if errors:
                    print(f"\n⚠️ {len(errors)} warning(s):")
                    for error in errors:
                        print(f"  • {error}")
        
        except Exception as e:
            with self.status_output:
                clear_output(wait=True)
                print(f"❌ Error during renaming: {e}")
                import traceback
                traceback.print_exc()
    
    def get_widget(self):
        """Return the complete widget"""
        return widgets.VBox([
            self.title,
            widgets.HTML("<hr style='margin: 10px 0;'>"),
            widgets.HTML("<b>1️⃣ Upload Files</b><br><i>Select CSV files or a ZIP file containing files to rename</i>"),
            self.file_uploader,
            self.file_list,
            self.clear_button,
            widgets.HTML("<b>2️⃣ Configure Renaming</b>"),
            self.kuerzel_input,
            self.date_input,
            self.auto_date_checkbox,
            self.prozessart_input,
            widgets.HTML("<b>3️⃣ Apply ELN Schema</b>"),
            self.rename_button,
            self.download_link,
            widgets.HTML("<b>Status:</b>"),
            self.status_output
        ], layout=widgets.Layout(
            padding='20px',
            border='2px solid #9c27b0',
            border_radius='8px'
        ))


print("✅ ELN Renamer Tool class loaded")

✅ ELN Renamer Tool class loaded


In [5]:
# UV-Vis Merger Tool
class UVVisMergerTool:
    """Tool for merging UV-Vis transmission and reflection data"""
    
    def __init__(self):
        self.transmission_files = {}
        self.reflection_files = {}
        self.manual_pairs = {}
        self.rename_enabled = False
        self.custom_names = {}
        
        self.trans_uploader = widgets.FileUpload(
            accept='.csv,.dat,.zip',
            multiple=True,
            description='Select T-files',
            layout=widgets.Layout(width='200px')
        )
        self.refl_uploader = widgets.FileUpload(
            accept='.csv,.dat,.zip',
            multiple=True,
            description='Select R-files',
            layout=widgets.Layout(width='200px')
        )
        
        self.create_widgets()
    
    def create_widgets(self):
        """Create UI widgets"""
        self.title = widgets.HTML(
            value="<h3>🌈 UV-Vis Data Merger</h3><p>Merge transmission and reflection spectroscopy data files</p>"
        )
        
        self.trans_uploader.observe(self._on_trans_upload, names='value')
        self.refl_uploader.observe(self._on_refl_upload, names='value')
        
        self.trans_list = widgets.HTML(value="<i>No transmission files</i>")
        self.refl_list = widgets.HTML(value="<i>No reflection files</i>")
        
        self.clear_trans_button = widgets.Button(description='Clear T-files', button_style='warning', icon='trash', layout=widgets.Layout(width='180px'))
        self.clear_refl_button = widgets.Button(description='Clear R-files', button_style='warning', icon='trash', layout=widgets.Layout(width='180px'))
        self.clear_trans_button.on_click(self._on_clear_trans)
        self.clear_refl_button.on_click(self._on_clear_refl)
        
        self.pairing_view = widgets.HTML(value="<i>Upload files to see pairings</i>")
        
        self.merge_button = widgets.Button(description='Merge Files', button_style='primary', icon='cog', layout=widgets.Layout(width='180px'), disabled=True)
        self.merge_button.on_click(self._on_merge)
        
        self.rename_button = widgets.ToggleButton(value=False, description='📝 Enable Rename', button_style='', icon='pencil', layout=widgets.Layout(width='180px'))
        self.rename_button.observe(self._on_rename_toggle, names='value')
        
        self.rename_section = widgets.HTML(value="")
        self.status_output = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='10px', min_height='80px', max_height='300px', overflow='auto'))
        self.download_link = widgets.HTML(value="")
    
    def _iter_uploaded_files(self, file_value):
        if not file_value:
            return []
        if isinstance(file_value, dict):
            return [
                (name, _ensure_bytes(info.get('content')))
                for name, info in file_value.items()
                if isinstance(info, dict)
            ]
        return [
            (info.get('name'), _ensure_bytes(info.get('content')))
            for info in file_value
            if isinstance(info, dict) and info.get('name')
        ]
    
    def _on_rename_toggle(self, change):
        """Handle rename toggle"""
        self.rename_enabled = change['new']
        if self.rename_enabled:
            self.rename_button.button_style = 'success'
            self.rename_button.description = '✓ Rename Enabled'
            self._update_rename_section()
        else:
            self.rename_button.button_style = ''
            self.rename_button.description = '📝 Enable Rename'
            self.rename_section.value = ""
            self.custom_names = {}
    
    def _update_rename_section(self):
        """Update the rename section"""
        if not self.rename_enabled:
            self.rename_section.value = ""
            return
        
        pairs, _, _ = self._find_matching_pairs()
        if not pairs:
            self.rename_section.value = "<div style='padding:15px; background:#fff3e0; border-radius:5px; margin:10px 0;'><i>Upload and pair files first to enable renaming</i></div>"
            return
        
        from datetime import datetime
        default_date = datetime.now().strftime("%Y%m%d")
        
        html = "<div style='padding:15px; background:#f0f4ff; border-radius:5px; margin:10px 0; border:2px solid #667eea;'>"
        html += "<h4 style='margin-top:0; color:#667eea;'>✏️ Custom File Naming</h4>"
        html += "<p style='color:#555; margin:5px 0 15px 0;'>Format: <code style='background:#fff; padding:3px 8px;'>PREFIX_NAME_DATE_FIELD1_FIELD2_FIELD3.SUFFIX</code></p>"
        html += "<div style='background:#fff; padding:15px; border-radius:5px; margin-bottom:15px;'><b>Global Settings (apply to all files):</b>"
        html += "<table style='width:100%; margin-top:10px;'><tr>"
        html += "<td style='padding:5px; width:15%;'><b>Prefix:</b></td><td style='padding:5px;'><input type='text' id='rename_prefix' value='KIT' oninput='updateRenamePreview()' style='width:100%; padding:5px; border:1px solid #ccc; border-radius:3px;'></td>"
        html += "<td style='padding:5px; width:15%;'><b>Name:</b></td><td style='padding:5px;'><input type='text' id='rename_name' placeholder='e.g. HaGu' oninput='updateRenamePreview()' style='width:100%; padding:5px; border:1px solid #ccc; border-radius:3px;'></td></tr><tr>"
        html += "<td style='padding:5px;'><b>Date:</b></td><td style='padding:5px;'><input type='text' id='rename_date' value='" + default_date + "' placeholder='YYYYMMDD' oninput='updateRenamePreview()' style='width:100%; padding:5px; border:1px solid #ccc; border-radius:3px;'></td>"
        html += "<td style='padding:5px;'><b>Suffix:</b></td><td style='padding:5px;'><input type='text' id='rename_suffix' value='.uvvis.csv' oninput='updateRenamePreview()' style='width:100%; padding:5px; border:1px solid #ccc; border-radius:3px;'></td></tr></table></div>"
        html += "<div style='background:#fff; padding:15px; border-radius:5px;'><b>Individual Settings (per file pair):</b><table style='width:100%; margin-top:10px; border-collapse:collapse;'>"
        html += "<tr style='background:#f5f5f5; border-bottom:2px solid #ddd;'><th style='padding:8px; text-align:left;'>File Pair</th><th style='padding:8px; text-align:center; width:13%;'>Field 1</th><th style='padding:8px; text-align:center; width:13%;'>Field 2</th><th style='padding:8px; text-align:center; width:13%;'>Field 3</th><th style='padding:8px; text-align:left; width:30%;'>Preview</th></tr>"
        
        for idx, (trans_file, refl_file) in enumerate(pairs.items()):
            row_color = '#fff' if idx % 2 == 0 else '#f9f9f9'
            html += f"<tr style='background:{row_color}; border-bottom:1px solid #eee;'><td style='padding:8px;'><small><code>{trans_file}</code> + <code>{refl_file}</code></small></td>"
            html += f"<td style='padding:5px;'><input type='text' id='field1_{idx}' placeholder='Field1' data-pair-idx='{idx}' data-trans='{trans_file}' oninput='updateRenamePreview()' style='width:100%; padding:4px; border:1px solid #ccc; border-radius:3px;'></td>"
            html += f"<td style='padding:5px;'><input type='text' id='field2_{idx}' placeholder='Field2' data-pair-idx='{idx}' oninput='updateRenamePreview()' style='width:100%; padding:4px; border:1px solid #ccc; border-radius:3px;'></td>"
            html += f"<td style='padding:5px;'><input type='text' id='field3_{idx}' placeholder='Field3' data-pair-idx='{idx}' oninput='updateRenamePreview()' style='width:100%; padding:4px; border:1px solid #ccc; border-radius:3px;'></td>"
            html += f"<td style='padding:8px;'><code id='preview_{idx}' style='color:#666; font-size:11px; word-break:break-all;'>KIT_NAME_{default_date}_FIELD1_FIELD2_FIELD3.uvvis.csv</code></td></tr>"
        
        html += "</table><button onclick='applyRenaming()' style='margin-top:15px; padding:8px 20px; background:#4caf50; color:white; border:none; border-radius:4px; cursor:pointer; font-weight:bold;'>✓ Apply Names</button></div></div>"
        
        script = '''
<script>
function updateRenamePreview(){
    const p=document.getElementById('rename_prefix').value||'KIT';
    const n=document.getElementById('rename_name').value||'NAME';
    const d=document.getElementById('rename_date').value||'DATE';
    const s=document.getElementById('rename_suffix').value||'.uvvis.csv';
    document.querySelectorAll('[id^="field1_"]').forEach(i=>{
        const x=i.dataset.pairIdx;
        const f1=document.getElementById('field1_'+x).value||'FIELD1';
        const f2=document.getElementById('field2_'+x).value||'FIELD2';
        const f3=document.getElementById('field3_'+x).value||'FIELD3';
        document.getElementById('preview_'+x).textContent=p+'_'+n+'_'+d+'_'+f1+'_'+f2+'_'+f3+s;
    });
}
function applyRenaming(){
    const p=document.getElementById('rename_prefix').value||'KIT';
    const n=document.getElementById('rename_name').value;
    const d=document.getElementById('rename_date').value;
    const s=document.getElementById('rename_suffix').value||'.uvvis.csv';
    if(!n||!d){alert('Please fill in Name and Date fields!');return;}
    const data={prefix:p,name:n,date:d,suffix:s,pairs:[]};
    document.querySelectorAll('[id^="field1_"]').forEach(i=>{
        const x=i.dataset.pairIdx;
        const t=i.dataset.trans;
        data.pairs.push({
            trans_file:t,
            field1:document.getElementById('field1_'+x).value||'',
            field2:document.getElementById('field2_'+x).value||'',
            field3:document.getElementById('field3_'+x).value||''
        });
    });
    const kernel=IPython.notebook.kernel;
    if(kernel){
        const json=JSON.stringify(data);
        const cmd=`import json; app.uvvis_merger._apply_rename_data(json.loads('` + json.replace(/'/g,"\\'") + `'))`;
        kernel.execute(cmd);
    }
}
updateRenamePreview();
</script>
'''
        html += script
        
        self.rename_section.value = html
    
    def _apply_rename_data(self, data):
        """Apply rename data from JavaScript"""
        self.custom_names = {}
        for pair in data['pairs']:
            trans_file = pair['trans_file']
            field1 = pair['field1'] or 'X'
            field2 = pair['field2'] or 'Y'
            field3 = pair['field3'] or 'Z'
            custom_name = f"{data['prefix']}_{data['name']}_{data['date']}_{field1}_{field2}_{field3}{data['suffix']}"
            self.custom_names[trans_file] = custom_name
        with self.status_output:
            clear_output()
            print(f"✅ Custom names applied for {len(self.custom_names)} file pair(s)")
            print("Ready to merge with new file names!")
    
    def _find_matching_pairs(self):
        """Find matching transmission and reflection file pairs"""
        pairs = {}
        unpaired_trans = set(self.transmission_files.keys())
        unpaired_refl = set(self.reflection_files.keys())
        
        for trans_file, refl_file in self.manual_pairs.items():
            if trans_file in self.transmission_files and refl_file in self.reflection_files:
                pairs[trans_file] = refl_file
                unpaired_trans.discard(trans_file)
                unpaired_refl.discard(refl_file)
        
        for trans_file in list(unpaired_trans):
            candidates = []
            if trans_file[-5:].lower() in ['t.csv', 't.dat']:
                candidate1 = trans_file[:-5] + trans_file[-5].replace('T', 'R').replace('t', 'r') + trans_file[-4:]
                if candidate1 in unpaired_refl:
                    candidates.append(candidate1)
            if '_T.' in trans_file or '_t.' in trans_file:
                candidate2 = trans_file.replace('_T.', '_R.').replace('_t.', '_r.')
                if candidate2 in unpaired_refl:
                    candidates.append(candidate2)
            for refl_file in unpaired_refl:
                if trans_file.replace('T', 'R').replace('t', 'r') == refl_file:
                    candidates.append(refl_file)
            if candidates:
                matched_refl = candidates[0]
                pairs[trans_file] = matched_refl
                unpaired_trans.discard(trans_file)
                unpaired_refl.discard(matched_refl)
        
        return pairs, unpaired_trans, unpaired_refl
    
    def _update_pairing_view(self):
        """Update the pairing visualization"""
        if not self.transmission_files and not self.reflection_files:
            self.pairing_view.value = "<i>Upload files to see pairings</i>"
            return
        
        pairs, unpaired_trans, unpaired_refl = self._find_matching_pairs()
        html = "<div><h4 style='margin-top:0; color:#333;'>📋 File Pairings</h4>"
        
        if pairs:
            html += "<div style='margin-bottom:20px;'><b style='color:#4caf50;'>✓ Matched Pairs:</b><table style='width:100%; border-collapse:collapse; margin-top:10px;'>"
            html += "<tr style='background:#f5f5f5; border-bottom:2px solid #ddd;'><th style='padding:8px; text-align:left;'>Transmission File</th><th style='padding:8px; text-align:center; width:120px;'>Action</th><th style='padding:8px; text-align:left;'>Reflection File</th></tr>"
            for idx, (trans_file, refl_file) in enumerate(pairs.items()):
                row_color = '#fff' if idx % 2 == 0 else '#f9f9f9'
                html += f"<tr style='background:{row_color}; border-bottom:1px solid #eee;'><td style='padding:8px;'><code style='background:#e3f2fd; padding:2px 6px; border-radius:3px;'>{trans_file}</code></td><td style='padding:8px; text-align:center;'>"
                html += f"<select id='pair_{idx}' onchange='window.updatePairing(\"{trans_file}\", this.value)' style='padding:4px; border-radius:3px; border:1px solid #ccc; width:100%;'><option value='{refl_file}' selected>{refl_file}</option>"
                for other_refl in self.reflection_files.keys():
                    if other_refl != refl_file:
                        html += f"<option value='{other_refl}'>{other_refl}</option>"
                html += "</select></td><td style='padding:8px;'><code style='background:#fff3e0; padding:2px 6px; border-radius:3px;'>{refl_file}</code></td></tr>"
            html += "</table></div>"
        
        if unpaired_trans:
            html += "<div style='margin-bottom:15px;'><b style='color:#ff9800;'>⚠️ Unpaired Transmission Files:</b><div style='margin-top:8px;'>"
            for idx, trans_file in enumerate(unpaired_trans):
                html += f"<div style='margin:5px 0; padding:8px; background:#fff3e0; border-left:4px solid #ff9800; border-radius:3px;'><code>{trans_file}</code><select id='unpaired_t_{idx}' onchange='window.updatePairing(\"{trans_file}\", this.value)' style='padding:4px; border-radius:3px; border:1px solid #ccc; margin-left:10px;'><option value=''>-- Select Reflection File --</option>"
                for refl_file in self.reflection_files.keys():
                    html += f"<option value='{refl_file}'>{refl_file}</option>"
                html += "</select></div>"
            html += "</div></div>"
        
        if unpaired_refl:
            html += "<div><b style='color:#ff9800;'>⚠️ Unpaired Reflection Files:</b><div style='margin-top:8px;'>"
            for refl_file in unpaired_refl:
                html += f"<div style='margin:5px 0; padding:8px; background:#ffebee; border-left:4px solid #f44336; border-radius:3px;'><code>{refl_file}</code> <span style='color:#666;'>(no matching transmission)</span></div>"
            html += "</div></div>"
        
        if pairs:
            html += f"<p style='margin-top:15px; color:#666; font-size:13px;'><b>Total:</b> {len(pairs)} pair(s) ready to merge"
            if unpaired_trans or unpaired_refl:
                html += f" | {len(unpaired_trans)} unpaired T-files | {len(unpaired_refl)} unpaired R-files"
            html += "</p>"
        
        html += "</div><script>window.updatePairing=function(transFile,reflFile){var kernel=IPython.notebook.kernel;if(kernel){var cmd=`app.uvvis_merger.manual_pairs['${transFile}']='${reflFile}' if '${reflFile}' else app.uvvis_merger.manual_pairs.pop('${transFile}',None); app.uvvis_merger._update_pairing_view(); app.uvvis_merger._update_rename_section()`;kernel.execute(cmd);}}</script>"
        self.pairing_view.value = html
    
    def _on_trans_upload(self, change):
        if not change['new']:
            return
        for filename, content in self._iter_uploaded_files(change['new']):
            if not filename:
                continue
            if filename.lower().endswith('.zip'):
                try:
                    with zipfile.ZipFile(io.BytesIO(content), 'r') as z:
                        for f in z.filelist:
                            if f.filename.lower().endswith(('.csv', '.dat')):
                                self.transmission_files[os.path.basename(f.filename)] = z.read(f)
                except Exception as e:
                    with self.status_output:
                        print(f"❌ Error reading zip file: {e}")
            elif filename.lower().endswith(('.csv', '.dat')):
                self.transmission_files[filename] = content
        self._update_file_lists()
        self._update_pairing_view()
        self._update_merge_button()
        if self.rename_enabled:
            self._update_rename_section()
    
    def _on_refl_upload(self, change):
        if not change['new']:
            return
        for filename, content in self._iter_uploaded_files(change['new']):
            if not filename:
                continue
            if filename.lower().endswith('.zip'):
                try:
                    with zipfile.ZipFile(io.BytesIO(content), 'r') as z:
                        for f in z.filelist:
                            if f.filename.lower().endswith(('.csv', '.dat')):
                                self.reflection_files[os.path.basename(f.filename)] = z.read(f)
                except Exception as e:
                    with self.status_output:
                        print(f"❌ Error reading zip file: {e}")
            elif filename.lower().endswith(('.csv', '.dat')):
                self.reflection_files[filename] = content
        self._update_file_lists()
        self._update_pairing_view()
        self._update_merge_button()
        if self.rename_enabled:
            self._update_rename_section()
    
    def _update_file_lists(self):
        if not self.transmission_files:
            self.trans_list.value = "<i>No transmission files</i>"
        else:
            items = [f"<li>{i}. {f}</li>" for i, f in enumerate(self.transmission_files.keys(), 1)]
            self.trans_list.value = f"<b>📁 Transmission files ({len(self.transmission_files)}):</b><ul style='margin-top: 5px;'>{''.join(items)}</ul>"
        if not self.reflection_files:
            self.refl_list.value = "<i>No reflection files</i>"
        else:
            items = [f"<li>{i}. {f}</li>" for i, f in enumerate(self.reflection_files.keys(), 1)]
            self.refl_list.value = f"<b>📁 Reflection files ({len(self.reflection_files)}):</b><ul style='margin-top: 5px;'>{''.join(items)}</ul>"
    
    def _update_merge_button(self):
        pairs, _, _ = self._find_matching_pairs()
        self.merge_button.disabled = len(pairs) == 0
    
    def _on_clear_trans(self, button):
        self.transmission_files = {}
        if isinstance(self.trans_uploader.value, dict):
            self.trans_uploader.value = {}
        else:
            self.trans_uploader.value = ()
        self.manual_pairs = {}
        self.custom_names = {}
        self._update_file_lists()
        self._update_pairing_view()
        self._update_merge_button()
        self._update_rename_section()
        self.download_link.value = ""
    
    def _on_clear_refl(self, button):
        self.reflection_files = {}
        if isinstance(self.refl_uploader.value, dict):
            self.refl_uploader.value = {}
        else:
            self.refl_uploader.value = ()
        self.manual_pairs = {}
        self.custom_names = {}
        self._update_file_lists()
        self._update_pairing_view()
        self._update_merge_button()
        self._update_rename_section()
        self.download_link.value = ""
    
    def _on_merge(self, button):
        with self.status_output:
            clear_output(wait=True)
            print("🔄 Merging files...")
        try:
            from uvvis_merger_module import process_uvvis_files
            pairs, unpaired_trans, unpaired_refl = self._find_matching_pairs()
            if not pairs:
                with self.status_output:
                    clear_output(wait=True)
                    print("⚠️ No file pairs found. Please check file names or manually pair files.")
                return
            zip_content, num_processed, errors = process_uvvis_files(
                {k: v for k, v in self.transmission_files.items() if k in pairs},
                {pairs[k]: self.reflection_files[pairs[k]] for k in pairs.keys()}
            )
            if self.rename_enabled and self.custom_names:
                original_zip = zipfile.ZipFile(io.BytesIO(zip_content), 'r')
                new_zip_buffer = io.BytesIO()
                with zipfile.ZipFile(new_zip_buffer, 'w', zipfile.ZIP_DEFLATED) as new_zip:
                    for original_name in original_zip.namelist():
                        matching_trans = None
                        for trans_file in pairs.keys():
                            base_trans = trans_file.rsplit('.', 1)[0].replace('_T', '').replace('_t', '').replace('T', '').replace('t', '')
                            if base_trans in original_name:
                                matching_trans = trans_file
                                break
                        new_name = self.custom_names[matching_trans] if matching_trans and matching_trans in self.custom_names else original_name
                        new_zip.writestr(new_name, original_zip.read(original_name))
                original_zip.close()
                new_zip_buffer.seek(0)
                zip_content = new_zip_buffer.read()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"merged_uvvis_files_{timestamp}.zip"
            b64_data = base64.b64encode(zip_content).decode()
            self.download_link.value = f'<div style="margin: 15px 0; padding: 15px; background: #e8f5e9; border-radius: 5px; border-left: 4px solid #4caf50;"><h4 style="margin-top: 0; color: #2e7d32;">✅ Merge Complete!</h4><p>Successfully merged <b>{num_processed}</b> file pairs.</p>{"<p><b>✏️ Custom file names applied</b></p>" if self.rename_enabled and self.custom_names else ""}<a href="data:application/zip;base64,{b64_data}" download="{filename}" style="display: inline-block; padding: 10px 20px; background: #4caf50; color: white; text-decoration: none; border-radius: 4px; font-weight: bold;">📥 Download {filename}</a></div>'
            with self.status_output:
                clear_output(wait=True)
                print(f"✅ Successfully merged {num_processed} file pairs")
                if self.rename_enabled and self.custom_names:
                    print("✏️ Custom file names applied")
                if unpaired_trans:
                    print(f"\n⚠️ {len(unpaired_trans)} transmission file(s) not merged (no matching reflection)")
                if unpaired_refl:
                    print(f"⚠️ {len(unpaired_refl)} reflection file(s) not merged (no matching transmission)")
                if errors:
                    print(f"\n⚠️ {len(errors)} file(s) had errors:")
                    for error in errors:
                        print(f"  • {error}")
                print("\n⬇️ Click the download button above to get your results")
        except Exception as e:
            with self.status_output:
                clear_output(wait=True)
                print(f"❌ Error during merge: {e}")
                import traceback
                traceback.print_exc()
    
    def get_widget(self):
        return widgets.VBox([
            self.title,
            widgets.HTML("<hr style='margin: 10px 0;'>"),
            widgets.HTML("<b>1️⃣ Upload Files</b>"),
            widgets.HTML("<i>Select transmission and reflection files:</i>"),
            widgets.HTML("<b>Transmission (T)</b>"),
            self.trans_uploader,
            self.trans_list,
            widgets.HTML("<b>Reflection (R)</b>"),
            self.refl_uploader,
            self.refl_list,
            widgets.HBox([self.clear_trans_button, self.clear_refl_button]),
            widgets.HTML("<b>2️⃣ Review and Adjust Pairings</b>"),
            widgets.HTML("<i>Files are automatically matched. Use dropdowns to manually adjust pairings:</i>"),
            self.pairing_view,
            widgets.HTML("<b>3️⃣ Optional: Custom File Names</b>"),
            self.rename_button,
            self.rename_section,
            widgets.HTML("<b>4️⃣ Merge and Download</b>"),
            self.merge_button,
            self.download_link,
            widgets.HTML("<b>Status:</b>"),
            self.status_output
        ], layout=widgets.Layout(
            padding='20px',
            border='2px solid #4caf50',
            border_radius='8px'
        ))

In [6]:
# Ratio Calculator Tool
class RatioCalculatorTool:
    """Tool for calculating mass and mole ratios"""
    
    def __init__(self):
        self.entries = []
        self.selected_button = None
        self.create_widgets()
    
    def create_widgets(self):
        """Create UI widgets"""
        self.title = widgets.HTML(
            value="<h3>⚖️ Ratio Calculator</h3><p>Calculate mass and (optional) mole ratios</p>"
        )
        
        # Header row
        header = widgets.HBox([
            widgets.HTML("<b>Name</b>", layout=widgets.Layout(width='200px')),
            widgets.HTML("<b>Mass (g)</b>", layout=widgets.Layout(width='140px')),
            widgets.HTML("<b>Molar mass (g/mol)</b>", layout=widgets.Layout(width='220px')),
            widgets.HTML("<b>Normalize to</b>", layout=widgets.Layout(width='130px'))
        ])
        
        self.rows_container = widgets.VBox()
        self.add_row()
        self.add_row()
        
        # Buttons
        self.add_button = widgets.Button(
            description='Add row',
            button_style='info',
            icon='plus',
            layout=widgets.Layout(width='210px')
        )
        self.add_button.on_click(self._on_add_row)
        
        self.delete_button = widgets.Button(
            description='Delete row',
            button_style='warning',
            icon='trash',
            layout=widgets.Layout(width='150px')
        )
        self.delete_button.on_click(self._on_delete_row)
        
        self.calculate_button = widgets.Button(
            description='Calculate',
            button_style='primary',
            icon='calculator',
            layout=widgets.Layout(width='150px')
        )
        self.calculate_button.on_click(self._on_calculate)
        
        self.status_output = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='10px'))
        self.result_html = widgets.HTML(value="")
        
        self.container = widgets.VBox([
            self.title,
            widgets.HTML("<hr style='margin: 10px 0;'>"),
            header,
            self.rows_container,
            widgets.HBox([self.add_button, self.delete_button, self.calculate_button]),
            widgets.HTML("<b>Status:</b>"),
            self.status_output,
            self.result_html
        ], layout=widgets.Layout(
            padding='20px',
            border='2px solid #607d8b',
            border_radius='8px'
        ))
    
    def _make_row(self):
        name_entry = widgets.Text(placeholder='Name', layout=widgets.Layout(width='200px'))
        mass_entry = widgets.Text(placeholder='e.g. 0.25', layout=widgets.Layout(width='140px'))
        molar_mass_entry = widgets.Text(placeholder='optional', layout=widgets.Layout(width='220px'))
        select_button = widgets.ToggleButton(
            value=False,
            icon='check',
            tooltip='Normalize to this material',
            layout=widgets.Layout(width='40px')
        )
        
        row = widgets.HBox([name_entry, mass_entry, molar_mass_entry, select_button])
        return (name_entry, mass_entry, molar_mass_entry, select_button, row)
    
    def _on_add_row(self, _):
        self.add_row()
    
    def add_row(self):
        entry = self._make_row()
        entry[3].observe(self._on_select, names='value')
        self.entries.append(entry)
        self.rows_container.children = [e[4] for e in self.entries]
    
    def _on_delete_row(self, _):
        if len(self.entries) <= 2:
            with self.status_output:
                clear_output()
                print("⚠️ At least two materials must remain.")
            return
        entry = self.entries.pop()
        if self.selected_button is entry[3]:
            self.selected_button = None
        entry[4].close()
        self.rows_container.children = [e[4] for e in self.entries]
    
    def _on_select(self, change):
        if change['new']:
            self.selected_button = change['owner']
            for entry in self.entries:
                btn = entry[3]
                if btn is self.selected_button:
                    btn.button_style = 'success'
                else:
                    if btn.value:
                        btn.value = False
                    btn.button_style = ''
        else:
            if change['owner'] is self.selected_button:
                self.selected_button = None
                change['owner'].button_style = ''
    
    def _on_calculate(self, _):
        with self.status_output:
            clear_output()
        self.result_html.value = ""
        
        names = []
        masses = []
        molar_masses = []
        valid_row_indices = []
        
        for idx, (name_entry, mass_entry, molar_mass_entry, _, _) in enumerate(self.entries):
            name = name_entry.value.strip()
            mass_text = mass_entry.value.strip()
            molar_text = molar_mass_entry.value.strip()
            
            if name and mass_text:
                try:
                    mass = float(mass_text)
                    if mass <= 0:
                        raise ValueError
                except ValueError:
                    with self.status_output:
                        print(f"❌ Invalid mass for '{name}'. Please enter a positive number.")
                    return
                names.append(name)
                masses.append(mass)
                valid_row_indices.append(idx)
                
                if molar_text:
                    try:
                        molar_mass = float(molar_text)
                        if molar_mass <= 0:
                            raise ValueError
                        molar_masses.append(molar_mass)
                    except ValueError:
                        with self.status_output:
                            print(f"❌ Invalid molar mass for '{name}'. Please enter a positive number.")
                        return
                else:
                    molar_masses.append(None)
        
        if not names:
            with self.status_output:
                print("⚠️ Please enter at least one valid row.")
            return
        
        selected_idx = None
        if self.selected_button is not None:
            for idx, entry in enumerate(self.entries):
                if entry[3] is self.selected_button:
                    selected_idx = idx
                    break
        
        if selected_idx is None or selected_idx not in valid_row_indices:
            with self.status_output:
                print("⚠️ Please select a material to normalize to.")
            return
        
        # Map selected index to valid list index
        selected_valid_index = valid_row_indices.index(selected_idx)
        
        # Mass ratios
        norm_mass = masses[selected_valid_index]
        normalized_mass_ratios = [round(m / norm_mass, 4) for m in masses]
        
        result_lines = []
        result_lines.append(f"<b>Normalized to {names[selected_valid_index]} = 1 (Mass ratio):</b><br>")
        for n, ratio in zip(names, normalized_mass_ratios):
            result_lines.append(f"{n}: {ratio}<br>")
        
        # Mole ratios if available
        if all(mm is not None for mm in molar_masses):
            moles = [m / mm for m, mm in zip(masses, molar_masses)]
            norm_moles = moles[selected_valid_index]
            normalized_mole_ratios = [round(nmol / norm_moles, 4) for nmol in moles]
            
            result_lines.append("<br><b>Normalized to {0} = 1 (Mole ratio):</b><br>".format(names[selected_valid_index]))
            for n, ratio in zip(names, normalized_mole_ratios):
                result_lines.append(f"{n}: {ratio}<br>")
        else:
            result_lines.append("<br><i>(Mole ratio not calculated — molar masses missing)</i><br>")
        
        self.result_html.value = "".join(result_lines)
    
    def get_widget(self):
        """Return the complete widget"""
        return self.container


print("✅ Ratio Calculator Tool class loaded")

✅ Ratio Calculator Tool class loaded


In [ ]:
# Main Application
class DataToolsApp:
    """Main application controller for Data Tools Dashboard"""
    
    def __init__(self):
        self.create_tools()
        self.create_ui()
    
    def create_tools(self):
        """Initialize all tools"""
        self.iv_converter = IVConverterTool()
        self.jv_organizer = JVOrganizerTool()
        self.eln_renamer = ELNRenamerTool()
        self.uvvis_merger = UVVisMergerTool()
        self.ratio_calculator = RatioCalculatorTool()
    
    def create_ui(self):
        """Create the main user interface"""
        # Header with Help button
        header = widgets.HTML(
            value="""
            <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                        padding: 20px; border-radius: 8px; margin-bottom: 20px; color: white; 
                        display: flex; justify-content: space-between; align-items: center;'>
                <div>
                    <h1 style='margin: 0; font-size: 28px;'>🛠️ Data Tools Dashboard</h1>
                    <p style='margin: 5px 0 0 0; font-size: 14px; opacity: 0.9;'>
                        JV/UV-Vis data processing and analysis tools
                    </p>
                </div>
                <button onclick="document.getElementById('helpModal').style.display='block'" 
                        style='padding: 10px 20px; background: rgba(255,255,255,0.2); border: 2px solid white; 
                               color: white; border-radius: 5px; cursor: pointer; font-size: 14px; font-weight: bold;
                               transition: background 0.3s;'
                        onmouseover="this.style.background='rgba(255,255,255,0.3)'"
                        onmouseout="this.style.background='rgba(255,255,255,0.2)'">
                    ℹ️ Help
                </button>
            </div>
            
            <!-- Help Modal -->
            <div id='helpModal' style='display:none; position:fixed; z-index:10000; left:0; top:0; 
                                       width:100%; height:100%; background-color:rgba(0,0,0,0.5);'>
                <div style='background-color:#fefefe; margin:5% auto; padding:30px; border:1px solid #888; 
                            border-radius:10px; width:90%; max-width:700px; max-height:80vh; overflow-y:auto;'>
                    <span onclick="document.getElementById('helpModal').style.display='none'" 
                          style='color:#aaa; float:right; font-size:28px; font-weight:bold; cursor:pointer;
                                 transition: color 0.3s;'
                          onmouseover="this.style.color='#000'"
                          onmouseout="this.style.color='#aaa'">&times;</span>
                    
                    <h2 style='color:#333; margin-top:0;'>📚 How to Use Data Tools Dashboard</h2>
                    
                    <h3 style='color:#667eea; margin-top:25px;'>📊 JV Converter Tool</h3>
                    <p style='color:#555; line-height:1.6;'>
                        Converts Puri JV measurement files from <code style='background:#f4f4f4; padding:2px 6px;'>_ivraw.csv</code> format 
                        to the old LTI format for compatibility with legacy systems.
                    </p>
                    <div style='background:#e3f2fd; padding:15px; border-radius:5px; margin:15px 0;'>
                        <b>Steps:</b>
                        <ol style='margin:10px 0;'>
                            <li>Click <b>"Upload Files"</b> and select your <code>_ivraw.csv</code> files or a ZIP archive</li>
                            <li>Review the uploaded files in the list</li>
                            <li>Click <b>"Convert Files"</b> to process the files</li>
                            <li>Download the converted ZIP file with <code>.px1.jv.csv, .px2.jv.csv</code>, etc.</li>
                        </ol>
                    </div>
                    <p style='color:#666; font-size:13px;'><b>Supported formats:</b> CSV files or ZIP archives containing multiple CSV files</p>
                    
                    <h3 style='color:#ff9800; margin-top:25px;'>📋 JV File Organizer</h3>
                    <p style='color:#555; line-height:1.6;'>
                        Organizes and renames JV measurement files by removing illumination suffixes, 
                        applying pixel naming schemes (_01_C → .px1_C), and organizing files into appropriate folders.
                    </p>
                    <div style='background:#fff3e0; padding:15px; border-radius:5px; margin:15px 0;'>
                        <b>Features:</b>
                        <ul style='margin:10px 0;'>
                            <li>Remove "_illu" suffix from filenames</li>
                            <li>Apply px naming scheme: _01_C → .px1_C, _02_C → .px2_C, etc.</li>
                            <li>Convert .csv to .jv.csv format</li>
                            <li>Organize files: MPP files → MaxPowerPointTracking folder, Soak files → Soak folder</li>
                            <li>Select which measurement cycle to keep (0-9)</li>
                            <li>Option to preserve cycle information in filenames</li>
                        </ul>
                    </div>
                    <p style='color:#666; font-size:13px;'><b>Input format:</b> CSV files with naming pattern like <code>Sample_01_C_illu.csv</code> or <code>Sample_Cycle_0_illu.csv</code></p>
                    
                    <h3 style='color:#9c27b0; margin-top:25px;'>📝 ELN File Renamer</h3>
                    <p style='color:#555; line-height:1.6;'>
                        Renames files according to the ELN naming schema for standardized documentation 
                        and organization of measurement files.
                    </p>
                    <div style='background:#f3e5f5; padding:15px; border-radius:5px; margin:15px 0;'>
                        <b>Features:</b>
                        <ul style='margin:10px 0;'>
                            <li>Rename files to ELN schema: <code style="background:#fff; padding:2px 5px;">KIT_{{kuerzel}}_{{datum}}_{{name}}_0_{{i}}.px{{pixel}}Cycle_{{cycle}}.{{prozessart}}.csv</code></li>
                            <li>Extract pixel and cycle information from filenames automatically</li>
                            <li>Support for custom researcher names (Kürzel): RaPe, DaBa, ThFe, etc.</li>
                            <li>Configurable date (YYYYMMDD format)</li>
                            <li>Process type specification (e.g., jv, eqe, abspl)</li>
                        </ul>
                    </div>
                    <p style='color:#666; font-size:13px;'><b>Example:</b> <code>Sample_01_Cycle_0_illu.csv</code> → <code>KIT_RaPe_20260119_Sample_0_0.px01Cycle_0.jv.csv</code></p>
                    
                    <h3 style='color:#ff6f00; margin-top:25px;'>🌈 UV-Vis Merger Tool</h3>
                    <p style='color:#555; line-height:1.6;'>
                        Automatically merges transmission (T) and reflection (R) spectroscopy data files by matching them 
                        based on filename patterns and aligning wavelength ranges.
                    </p>
                    <div style='background:#fff3e0; padding:15px; border-radius:5px; margin:15px 0;'>
                        <b>Steps:</b>
                        <ol style='margin:10px 0;'>
                            <li>Upload <b>Transmission files</b> (ending with T or t)</li>
                            <li>Upload <b>Reflection files</b> (ending with R or r)</li>
                            <li>The tool automatically pairs files with matching names</li>
                            <li>Optionally customize output file names using the Rename feature</li>
                            <li>Click <b>"Merge Files"</b> to combine the data</li>
                            <li>Download the merged ZIP file with combined T+R data</li>
                        </ol>
                    </div>
                    <p style='color:#666; font-size:13px;'><b>Naming convention:</b> Files are matched by replacing the last character (T/R) 
                    or suffix (_T/_R). Example: <code>sample_T.csv</code> matches <code>sample_R.csv</code></p>
                    
                    <h3 style='color:#607d8b; margin-top:25px;'>⚖️ Ratio Calculator</h3>
                    <p style='color:#555; line-height:1.6;'>
                        Calculates mass ratios and (optional) mole ratios from simple input tables. 
                        Enter names, masses, and optionally molar masses, then choose a reference material for normalization.
                    </p>
                    <div style='background:#eceff1; padding:15px; border-radius:5px; margin:15px 0;'>
                        <b>Steps:</b>
                        <ol style='margin:10px 0;'>
                            <li>Enter at least two materials with mass values</li>
                            <li>Optionally provide molar masses for all rows</li>
                            <li>Select the reference material in the table</li>
                            <li>Click <b>"Calculate"</b> to see mass and mole ratios</li>
                        </ol>
                    </div>
                    
                    <h3 style='color:#333; margin-top:25px;'>💡 Tips</h3>
                    <ul style='color:#555; line-height:1.8;'>
                        <li>Use <b>ZIP files</b> to upload multiple files at once</li>
                        <li>Click <b>"Clear"</b> buttons to remove files and start over</li>
                        <li>Check the <b>Status</b> section for processing messages and errors</li>
                        <li>All conversions are done locally - your data is not uploaded anywhere</li>
                    </ul>
                    
                    <div style='margin-top:20px; padding:15px; background:#f5f5f5; border-left:4px solid #4caf50; border-radius:5px;'>
                        <p style='color:#2e7d32; margin:0;'><b>✅ Ready to get started?</b> Close this dialog and select a tool above!</p>
                    </div>
                </div>
            </div>
            
            <script>
                // Close modal when clicking outside of it
                window.onclick = function(event) {
                    var modal = document.getElementById('helpModal');
                    if (event.target == modal) {
                        modal.style.display = 'none';
                    }
                }
            </script>
            """
        )
        
        # Info box
        info_box = widgets.HTML(
            value="""
            <div style='background: #e3f2fd; padding: 15px; border-radius: 5px; 
                        border-left: 4px solid #2196f3; margin-bottom: 20px;'>
                <b>ℹ️ Select a tool:</b>
                <ul style='margin: 10px 0 0 20px; padding: 0;'>
                    <li><b>📊 JV Tandem split:</b> Split Puri JV measurements into LTI format</li>
                    <li><b>📋 JV Organizer:</b> Rename and organize JV measurement files</li>
                    <li><b>📝 ELN Renamer:</b> Rename files according to ELN naming schema</li>
                    <li><b>🌈 UV-Vis Merger:</b> Merge transmission and reflection spectroscopy data</li>
                    <li><b>⚖️ Ratio Calculator:</b> Calculate mass and mole ratios</li>
                </ul>
                <p style='margin: 10px 0 0 0; font-size: 13px; color: #666;'>
                    Click the <b>ℹ️ Help</b> button in the header for detailed instructions.
                </p>
            </div>
            """
        )
        
        # Create tabs for the five tools
        self.tabs = widgets.Tab()
        self.tabs.children = [
            self.iv_converter.get_widget(),
            self.jv_organizer.get_widget(),
            self.eln_renamer.get_widget(),
            self.uvvis_merger.get_widget(),
            self.ratio_calculator.get_widget()
        ]
        
        self.tabs.set_title(0, '📊 JV Tandem split')
        self.tabs.set_title(1, '📋 JV Organizer')
        self.tabs.set_title(2, '📝 ELN Renamer')
        self.tabs.set_title(3, '🌈 UV-Vis Merger')
        self.tabs.set_title(4, '⚖️ Ratio Calculator')
        
        # Footer
        footer = widgets.HTML(
            value="""
            <div style='text-align: center; margin-top: 30px; padding: 15px; 
                        color: #666; font-size: 12px; border-top: 1px solid #ddd;'>
                <p style='margin: 0;'>Data Tools Dashboard | KIT | 2026</p>
                <p style='margin: 5px 0 0 0;'>For questions or issues, contact your lab administrator</p>
            </div>
            """
        )
        
        # Main container
        self.container = widgets.VBox([
            header,
            info_box,
            self.tabs,
            footer
        ], layout=widgets.Layout(
            max_width='1200px',
            margin='0 auto',
            padding='20px'
        ))
    
    def get_dashboard(self):
        """Return the complete dashboard"""
        return self.container


print("✅ DataToolsApp class loaded")

✅ DataToolsApp class loaded


In [ ]:
# Launch the application
try:
    # Set document title for browser tab
    display(HTML("<script>document.title='Data Tools Dashboard'</script>"))
    
    # Try to log usage if access_token module is available
    try:
        sys.path.append(os.path.dirname(os.getcwd()))
        import access_token
        access_token.log_notebook_usage()
    except ImportError:
        pass
    
    # Create and display the app
    app = DataToolsApp()
    display(app.get_dashboard())
    
    print("\n✅ Data Tools Dashboard launched successfully!")
    print("📝 Note: This app works in both Jupyter and Voilà mode")
    
except Exception as e:
    print(f"❌ Error launching application: {e}")
    import traceback
    traceback.print_exc()


✅ Data Tools Dashboard launched successfully!
📝 Note: This app works in both Jupyter and Voilà mode
